<a href="https://colab.research.google.com/github/AriKlein/intro_rl_foosball_lec2/blob/main/W3S1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning with Foolsball
- Reinforcement learning is learning to make decisions from experience.
- Games are a good testbed for RL.
 

# About Foolsball
- 5x4 playground that provides a football/foosball-like environment.
- A controllable player:
  - always spawned in the top-left corner
  - displayed as '⚽'
  - can move North, South, East or West.
  - can be controlled algorithmically
- A number of **static** opponents, each represented by 👕, that occupy certain locations on the field.
- A goalpost 🥅 that is fixed in the bottom right corner

## Goals
### Primary goal
- We want the agent to learn to reach the goalpost 

### Secondary goals
- We may want the agent to learn to be efficient in some sense, for example, take the shortest path to the goalpost. 

## Rules 
- Initial rules:
    - The ball can be (tried to be) moved in four direction: \['n','e','w',s'\]
    - Move the ball to an unmarked position: -1 points
    - Move the ball to a position marked by a defender: -5 points
    - Try to move the ball ouside the field: -1 (ball stays in the previous position)
    - Move the ball into the goal post position: +5


# Create the enviroment

In [ ]:
import numpy as np

agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

In [ ]:
class Foolsball(object):
    def __to_state__(self,row,col):
        """Convert from indices (row,col) to integer position."""
        return row*self.n_cols + col
    
    
    def __to_indices__(self, state):
        """Convert from inteeger position to indices(row,col)"""
        row = state // self.n_cols
        col = state % self.n_cols
        return row,col

    def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
        """Convrt a string representation of a map into a 2D numpy array
        Param map: list of lists of strings representing the player, opponents and goal.
        Param agent: string representing the agent on the map 
        Param opponent: string representing every instance of an opponent player
        Param goal: string representing the location of the goal on the map
        """
        ## Capture dimensions and map.
        self.n_rows = len(map)
        self.n_cols = len(map[0])
        self.n_states = self.n_rows * self.n_cols
        self.map = np.asarray(map)

        ## Store string representations for printing the map, etc.
        self.agent_repr = agent
        self.opponent_repr  = opponent
        self.goal_repr = goal

        ## Find initial state, the desired goal state and the state of the opponents. 
        self.init_state = None
        self.goal_state = None
        self.opponents_states = []

        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if map[row][col] == agent:
                    # Store the initial state outside the map.
                    # This helps in quickly resetting the game to the initial state and
                    # also simplifies printing the map independent of the agent's state. 
                    self.init_state = self.__to_state__(row,col)
                    self.map[row,col] = ' ' 

                elif map[row][col] == opponent:
                    self.opponents_states.append(self.__to_state__(row,col))

                elif map[row][col] == goal:
                    self.goal_state = self.__to_state__(row,col)

        assert self.init_state is not None, f"Map {map} does not specify an agent {agent} location"
        assert self.goal_state is not None,  f"Map {map} does not specify a goal {goal} location"
        assert self.opponents_states,  f"Map {map} does not specify any opponents {opponent} location"

        return self.init_state
    
    
    def __init__(self,map,agent,opponent,goal):
        """Spawn the world, create variables to track state and actions."""
        # We just need to track the location of the agent (the ball)
        # Everything else is static and so a potential algorithm doesn't 
        # have to look at it. The variable `done` flags terminal states.
        self.state = self.__deserialize__(map,agent,opponent,goal)
        self.done = False
        self.actions = ['n','e','w','s']

        # Set up the rewards
        self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
        self.set_rewards(self.default_rewards)
        
    def set_rewards(self,rewards):
        if not self.state == self.init_state:
            print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
        for key in self.default_rewards:
            assert key in rewards, f'Key {key} missing from reward.'
        self.rewards = rewards
            
            
    def reset(self):
        """Reset the environment to its initial state."""
        # There's really just two things we need to reset: the state, which should
        # be reset to the initial state, and the `done` flag which should be 
        # cleared to signal that we are not in a terminal state anymore, even if we 
        # were earlier. 
        self.state = self.init_state
        self.done  = False
        return self.state
    
    def __get_next_state_on_action__(self,state,action):
        """Return next state based on current state and action."""
        assert not self.__is_terminal_state__(state), f"Action {action} undefined for terminal state {state}"
        
        row, col = self.__to_indices__(state)
        action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

        row_delta, col_delta = action_to_index_delta[action]
        new_row , new_col = row+row_delta, col+col_delta

        ## Return current state if next state is invalid
        if not(0<=new_row<self.n_rows) or\
        not(0<=new_col<self.n_cols):
            return state  

        ## Construct state from new row and col and return it.    
        return self.__to_state__(new_row, new_col)    
    
  
    def __get_reward_for_transition__(self,state,next_state):
        """ Return the reward based on the transition from current state to next state. """
        ## Transition rejected due to illegal action (move)
        assert not self.__is_terminal_state__(state), f"Reward is undefined for terminal state {state}"
        
        if next_state == state:
            reward = self.rewards['outside']

        ## Goal!
        elif next_state == self.goal_state:
            reward = self.rewards['goal']

        ## Ran into opponent. 
        elif next_state in self.opponents_states:
            reward = self.rewards['opponent']

        ## Made a safe and valid move.   
        else:
            reward = self.rewards['unmarked']

        return reward    
    
    
    def __is_terminal_state__(self, state):
        return (state == self.goal_state) or (state in self.opponents_states) 
    
      
    def step(self,action):
        """Simulate state transition based on current state and action received."""
        assert not self.done, \
        f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.'
        next_state = self.__get_next_state_on_action__(self.state, action)

        reward = self.__get_reward_for_transition__(self.state, next_state)

        done = self.__is_terminal_state__(next_state)

        self.state, self.done = next_state, done

        return next_state, reward, done
    
    
    
    def render(self):
        """Pretty-print the environment and agent."""
        ## Create a copy of the map and change data type to accomodate
        ## 3-character strings
        _map = np.array(self.map, dtype='<U3')

        ## Mark unoccupied positions with special symbol.
        ## And add extra spacing to align all columns.
        for row in range(_map.shape[0]):
            for col in range(_map.shape[1]):
                if _map[row,col] == ' ':
                    _map[row,col] = ' + '

                elif _map[row,col] == self.opponent_repr: 
                    _map[row,col] =  self.opponent_repr + ' '

                elif _map[row,col] == self.goal_repr:
                    _map[row,col] = ' ' + self.goal_repr + ' '

        ## If current state overlaps with the goal state or one of the opponents'
        ## states, susbstitute a distinct marker.
        if self.state == self.goal_state:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' 🏁 '
        elif self.state in self.opponents_states:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' ❗ '
        else:
            r,c = self.__to_indices__(self.state)
            _map[r,c] = ' ' + self.agent_repr

        for row in range(_map.shape[0]):
            for col in range(_map.shape[1]):
                print(f' {_map[row,col]} ',end="")
            print('\n') 

        print()


In [ ]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [ ]:
foolsball.render()

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  




# Override the default reward structure.
- Use a more sparse reward: {'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5}

In [ ]:
## Update reward structure to: {'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5}
foolsball.reset()
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

# Implement discounted returns¶
$$Discounted\ Return = R_{t_1} + \gamma*R_{t_2} + \gamma^2*R_{t_3} + ... + \gamma^{n-1}*R_{t_n}$$where $R_{t_k}$ is the reward after step k and $\gamma$ is called the discount factor.
- Set the discount factor $\gamma$ to 0.9

In [ ]:
def get_discounted_return(path, gamma=0):
    foolsball.reset()
    foolsball.render()
    _return_ = 0
    discount_coeff = 1
    for act in path: 
        next_state, reward, done = foolsball.step(act)
        _return_ += discount_coeff*reward
        discount_coeff *= gamma    

        foolsball.render()
        if done:
            break
            
    print(f'Return (accumulated reward): {_return_}')

In [ ]:
HYPER_PARAMS = {'gamma':0.9}

# Use dynamic programming to fill up the returns table
- The **highest discounted return** for a **(state, action)** can be defined in terms of returns of the next state.

$ Return(state_t,action_t) = Reward(state_t,state_{t+1}) + \gamma * \max \begin{bmatrix} Return(state_{t+1}, action_{t+1}=='n')\\ Return(state_{t+1}, action_{t+1}=='e')\\  Return(state_{t+1}, action_{t+1}=='w')\\  Return(state_{t+1}, action_{t+1}=='s') \end{bmatrix}$


In [ ]:
import pandas as pd
def make_returns_table(states_list, actions_list, terminal_states):
    """Create an empty returns table where each entry is initialized arbitrarily."""
    table = pd.DataFrame.from_dict({s:{a:0 for a in actions_list} for s in states_list}, orient='index')
    return table

In [ ]:
terminal_states = foolsball.opponents_states + [foolsball.goal_state]
RETURNS_TBL = make_returns_table(range(foolsball.n_states), foolsball.actions, terminal_states)
RETURNS_TBL

,n,e,w,s
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [ ]:
def compute_returns(table,state,action, debug=False): 
    """ Recursively compute the discounted return for a (state,action) pair"""
    if not foolsball.__is_terminal_state__(state):

        next_state = foolsball.__get_next_state_on_action__(state, action)
        reward = foolsball.__get_reward_for_transition__(state, next_state)

        update = HYPER_PARAMS['gamma'] *\
        max(table.loc[next_state, foolsball.actions[0]],\
        table.loc[next_state, foolsball.actions[1]],\
        table.loc[next_state, foolsball.actions[2]],\
        table.loc[next_state, foolsball.actions[3]])

        table.loc[state, action]  = reward + update
    
    return table.loc[state,action]

In [ ]:
for s in range(foolsball.n_states):
    for a in foolsball.actions:
        compute_returns(RETURNS_TBL,state=s, action=a, debug=True)
RETURNS_TBL

,n,e,w,s
0,-1.0,0.0,-1.0,0.0
1,-1.0,-5.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,-1.0,-1.0,-5.0,-5.0
4,0.0,0.0,-1.0,0.0
5,0.0,0.0,0.0,-5.0
6,-5.0,-5.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,-5.0,-1.0,0.0
9,0.0,0.0,0.0,0.0


# Let the returns stabilize (converge)¶

In [ ]:
RETURNS_TBL = make_returns_table(range(foolsball.n_states), foolsball.actions, terminal_states)
for i in range(1,50):
    RETURNS_TBL_OLD = RETURNS_TBL.copy()
    for s in range(foolsball.n_states):
        for a in foolsball.actions:
            compute_returns(RETURNS_TBL,state=s, action=a, debug=True)
    
    if i%5 == 0:
        print(f'\n{i} iterations')
        print(RETURNS_TBL)
    
    deltas = RETURNS_TBL- RETURNS_TBL_OLD
    if abs(deltas.values.max()) < 1e-3:
        print(f'\nConvergence achieved at {i} iterations')
        print(RETURNS_TBL)
        break


5 iterations
          n       e        w        s
0  -1.00000  0.0000 -1.00000  0.00000
1  -1.00000 -5.0000  0.00000  0.00000
2   0.00000  0.0000  0.00000  0.00000
3  -4.09510 -4.0951 -5.00000 -5.00000
4   0.00000  0.0000 -1.00000  0.00000
5   0.00000  3.2805  0.00000 -5.00000
6  -5.00000 -5.0000  2.95245  3.64500
7   0.00000  0.0000  0.00000  0.00000
8   0.00000 -5.0000 -1.00000  3.28050
9   0.00000  0.0000  0.00000  0.00000
10  3.28050  3.2805 -5.00000  4.05000
11 -5.00000  2.2805  3.64500 -5.00000
12  2.95245  3.6450  2.28050  2.95245
13 -5.00000  4.0500  3.28050 -5.00000
14  3.64500 -5.0000  3.64500  4.50000
15  0.00000  0.0000  0.00000  0.00000
16  3.28050 -5.0000  1.95245  1.95245
17  0.00000  0.0000  0.00000  0.00000
18  4.05000  5.0000 -5.00000  3.50000
19  0.00000  0.0000  0.00000  0.00000

Convergence achieved at 9 iterations
           n         e         w         s
0   1.391485  2.657205  1.391485  2.657205
1   1.657205 -5.000000  2.391485  2.952450
2   0.000000  0.00000

# Intro to Policies

In [ ]:
def greedy_policy_from_returns_tbl(table):
    policy = {s:None for s in table.index }
    for state in table.index:
        if state not in terminal_states:
            greedy_action = table.loc[state].idxmax()
            policy[state] = greedy_action
            
    return policy

In [ ]:
policy0 = greedy_policy_from_returns_tbl(RETURNS_TBL)
policy0

{0: 'e',
 1: 's',
 2: None,
 3: 'w',
 4: 'e',
 5: 'e',
 6: 's',
 7: None,
 8: 's',
 9: None,
 10: 's',
 11: 'w',
 12: 'e',
 13: 'e',
 14: 's',
 15: None,
 16: 'n',
 17: None,
 18: 'e',
 19: None}

In [ ]:
def pretty_print_policy(policy):
    direction_repr = {'n':' 🡑 ', 'e':' 🡒 ', 'w':' 🡐 ', 's':' 🡓 ', None:' ⬤ '}

    for row in range(foolsball.n_rows):
        for col in range(foolsball.n_cols):
            state = foolsball.__to_state__(row, col)
            print(direction_repr[policy[state]],end='')
        print()

In [ ]:
pretty_print_policy(policy0)

 🡒  🡓  ⬤  🡐 
 🡒  🡒  🡓  ⬤ 
 🡓  ⬤  🡓  🡐 
 🡒  🡒  🡓  ⬤ 
 🡑  ⬤  🡒  ⬤ 


# Dealing with incomplete Knowledge of the environment
- We may not know all the states of the environment in advance
- We may not know the single-step dynamics $state_{t+1} | (state_{t},\ action_{t})$
- In terms of code, we cannot use the private method : `__get_next_state_on_action__()` ,as in 

```{python}
next_state = foolsball.__get_next_state_on_action__(state, action)
```

- We can attempt to learn these unknows from experience(sampling). 

## Todo
- Implement the function `collect_random_episode()` and run the code in the next cell to collect and print a random episode.

    - The episode starts with the environment in the initial state
    - The agent tries random actions
    - The episode terminates when the agent collides with an opponent or reaches the goalpost.


In [ ]:
import numpy as np
def collect_random_episode():
    state = foolsball.reset()
    done = False
    episode = []
    
    while not done:
        action = np.random.choice(foolsball.actions)
        next_state, reward, done = foolsball.step(action)
        episode.append([state, action, reward])
        state = next_state
        
    return episode

In [ ]:
ep = collect_random_episode()
foolsball.render()
print(ep)

  +    +    ❗    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


[[0, 'e', 0], [1, 'n', -1], [1, 'w', 0], [0, 'n', -1], [0, 'e', 0], [1, 'e', -5]]


# Step 10: Implement discounted returns for episodes

- Complete the function `discounted_return_from_episode()` that computes the discounted return for every (state,action) pair in an episode.
- If an episode is: (s1,a1,r1),(s2,a2,r2),(s3,a3,r3), (s4),  s4 being a terminal state:
  - The (discounted) return for (s1,a1) is r1+γ∗r2+γ2∗r3
  - The (discounted) return for (s2, a2)is r2+γ∗r3
  - The (discounted) return for (s3,a3) is r3

- Run the next cell to print discounted returns for entire episodes.


In [ ]:
def discounted_return_from_episode(ep, gamma=0):
    states, actions, rewards = list(zip(*ep))
    rewards = np.asarray(rewards)
    discount_coeffs = np.asarray([np.power(gamma,p) for p in range(len(rewards))])
    
    l = len(rewards)
    discounted_returns = [np.dot(rewards[i:],discount_coeffs[:l-i]) for i in range(l)]
    
    return (states, actions, discounted_returns)

In [ ]:
discounted_return_from_episode(ep, gamma=HYPER_PARAMS['gamma'])

((0, 1, 1, 0, 0, 1),
 ('e', 'n', 'w', 'n', 'e', 'e'),
 [-4.58145,
  -5.0905000000000005,
  -4.545000000000001,
  -5.050000000000001,
  -4.5,
  -5.0])

# Step 11: Estimate returns by simulating lots of episodes.

- The code below creates two tables:
  - ESTIMATED_RETURNS_TBL for accumulating the return for every (state,action) pair across all episodes.
  - VISITS_COUNTS_TBL for storing the number of times a (state,action) pair appears across all episodes.

- It then runs an algorithm to generate episodes and fill in these tables.

Here's the idea:

- Generate many random episodes
  - Examine each (state, action) pair in every episode.
  - Calculate and accumulate the return for this pair
     - Since we have the full episode, we can "see the future" and calculate the return.
     - The return for a (state,action) pair is just a (very bad) estimate of the "real" return, since we are looking at just one of the many paths that could possible contain the (state,action).
  - Record the visit count of the (state, action) pair.

- At the end the we divide the accumulated returns by the visit counts to get an averaged estimate of the retruns.

## Todo:

- Complete the code in the next cell to implement what's known as **Monte Carlo estimation**.
- Run the cells to see how well the algorithm fares.
- Does the algorithm generate sensible looking returns (estimates)?



In [ ]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 100  #Try 100, 500, 2000

for i in range(n_episodes):
    episode_i = collect_random_episode()
    states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

    for s,a,ret in zip(states, actions, discounted_returns):
        ESTIMATED_RETURNS_TBL.loc[s,a] += ret
        VISITS_COUNTS_TBL.loc[s,a] += 1

In [ ]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-4.993052,-3.967843,-4.900700,-3.605963
1,-4.985317,-4.868421,-4.015478,-3.659481
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-4.131873,-3.809256,-4.680268,-3.727930
5,-3.836785,-3.796222,-3.903569,-4.814815
6,-4.166667,-4.375000,-3.451337,-2.910332
7,0.000000,0.000000,0.000000,0.000000
8,-3.543223,-4.722222,-4.497331,-3.500620
9,0.000000,0.000000,0.000000,0.000000


In [ ]:
policy1 = greedy_policy_from_returns_tbl(estimated_returns)
pretty_print_policy(policy1)

 🡓  🡓  ⬤  🡑 
 🡓  🡒  🡓  ⬤ 
 🡓  ⬤  🡐  🡒 
 🡒  🡑  🡑  ⬤ 
 🡐  ⬤  🡒  ⬤ 


# Step 12: Exploiting the information in the returns table.

- We are improving our estimates of the returns with each successive episode.
- But we are still generating random episodes throughout.
- We should also exploit the information we accrue in the RETURNS table.

- Complete the implementation of `collect_greedy_episode_from_returns_tbl()` below which should be quite similar to `collect_random_episode()` but with the following key difference:
   - In state s, collect_random_episode() was returning a random action from ('n', 's', 'e', 'w').
   - But from the returns table we know that one of the action, say 'e' generates the best returns so we can make a greedy choice and always return 'e'.
   - This is what collect_greedy_episode_from_returns_tbl() should do.

- Run the next to cells to see the difference.

In [ ]:
def collect_greedy_episode_from_returns_tbl(table, max_ep_len=20):
    episode = []
    state = foolsball.reset()
    done = False
  
    for _ in range(max_ep_len):
        if done:
            break

        greedy_action = table.loc[state].idxmax()
        next_state, reward, done = foolsball.step(greedy_action)
        episode.append([state, greedy_action, reward])
        state = next_state
        #todo
    return episode

In [ ]:
collect_greedy_episode_from_returns_tbl(estimated_returns)

[[0, 's', 0], [4, 's', 0], [8, 's', 0], [12, 'e', 0], [13, 'n', -5]]

# Step 13: Updating the Returns Table Using Greedy Episodes
## Todo

- Implement the missing code below to update the returns table.

- The code will be exactly what we used earlier, except that it will use greedy episodes.

- Run the next few cells to evaluate the effectiveness.



In [ ]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
    episode_i = collect_greedy_episode_from_returns_tbl(ESTIMATED_RETURNS_TBL)
    states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

    for s,a,ret in zip(states, actions, discounted_returns):
        ESTIMATED_RETURNS_TBL.loc[s,a] += ret
        VISITS_COUNTS_TBL.loc[s,a] += 1
    #Todo

In [ ]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-5.759138,-3.892117,-5.759138,0.0
1,-5.607883,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.0


In [ ]:
policy2 = greedy_policy_from_returns_tbl(estimated_returns)

In [ ]:
pretty_print_policy(policy2)

 🡓  🡒  ⬤  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  ⬤ 


# Step 14: The Exploration-exploitation Dilemma¶

- We have tried pure exploration (with random episodes)
- We have also tried pure exploitation (with policy generated from the returns table)
- A good agent should try to balance both.

## Epsilon-greedy episodes

- An epsilon greedy episode blends the previous two approaches

- Precisely, when in state s:
  - The epsilon greedy episode will pick the action yielding the highest returns with a high probability, say 0.8
  - With a low probability it can return one of the suboptimal, random actions.
  - The hyperparameter epsilon or ϵ decides the probability of selecting a random action and 1-epsilon is the probability of picking the best action.

  - Example with epsilon = 0.2 and assuming w is the best action
    - state s
    - Actions = ('n','e','w','s')
    - Best action (yielding highest return) = 'w'
    - Sampling probabilities = [ϵ4,ϵ4,1−ϵ+ϵ4,ϵ4]=[0.05,0.05,0.85,0.05]

## Todo:

Finish the code below and look at how the output differs from the other two methods.

In [ ]:
def collect_epsilon_greedy_episode_from_returns_tbl(table, max_ep_len=20, epsilon=0.1):
    state = foolsball.reset()
    done = False
    episode = []
    
    for _ in range(max_ep_len):
        if done:
            break
        
        #Create a list/array of epsilon-greedy probabilities for all actions

        # Sample an action using the previously-commputed probabilities.
    
        # Simulate the action, update episode and state

    return episode

In [ ]:
# Generate an epsilon-greedy episode every time. 
collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns, epsilon=1)

# Step 15: Updating the Returns Table Using Epsilon-greedy Episodes.
Todo:
    - Run the next few cells to see the effect of using an epsilon greedy approach

In [ ]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
    estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
    episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns)
    
    states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])
    
    for s,a,ret in zip(states, actions, discounted_returns):
        ESTIMATED_RETURNS_TBL.loc[s,a] += ret
        VISITS_COUNTS_TBL.loc[s,a] += 1

In [ ]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

In [ ]:
policy3 = greedy_policy_from_returns_tbl(estimated_returns)
policy3

In [ ]:
pretty_print_policy(policy3)

# Step 16: Revisiting Exploration-Exploitation with Epsilon Decay
- What is the best way to balance exploitation with exploration?
    - In the beginning, pick absolutely random actions in every state.
    - Slowly reduce the randomness to a small value.

Todo:
   - In the code below pick a value of epsilon that makes all actions equiprobable in collect_epsilon_greedy_episode_from_returns_tbl().
   - Fill in the code to 'anneal' epsilon over episodes. The value of epsilon shoud not drop below the minimum threshold.
   - Run the next few cells to evaluate this approach.
   - Does the policy look any better?

In [ ]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

for i in range(n_episodes):
    # Implement table updation using epsilon decay: reduce the value of epsilon every iteration 
    # until it reaches min_epsilon. 

In [ ]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
print(estimated_returns)

policy4 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy4)

pretty_print_policy(policy4)

# Step 17: Constant Alpha

## The idea:
- Dividing the accumulated returns by visit count has a non linear effect on the updates. (Go back to previous step and see for yourself).
- Don't divide at all!
- But we need to ensure that updates are small
- Idea:
 - ESTIMATED_RETURNS_TBL.loc[s,a] and ret are both estimates of the same quantity.
 - Use the difference of the two estimates to update ESTIMATED_RETURNS_TBL.loc[s,a] much like we do in Deep Learning.

Todo:
- Complete the missing code in the next cell.
- Run the next few cells to get a policy and evaluate it.
- Does the policy help the agent attain its goal?



In [ ]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.01

for i in range(n_episodes):
    #Todo: Implement updates using constant alpha

In [ ]:
estimated_returns = ESTIMATED_RETURNS_TBL
print(estimated_returns)

policy5 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy5)

pretty_print_policy(policy5)